In [15]:

from skimage import transform
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns


In [26]:
data_folder = 'data/cubicasa5k/'
data_file = 'val.txt'
training_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
rooms = ["Outdoor", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Storage", "Garage", "Undefined"]
room_ids = [1,3,4,5,6,7,9,10,11]
room_labels = {rooms[i]:i+1 for i in range(len(rooms))}

In [28]:
import copy
import cv2
from tqdm import tqdm_notebook as tqdm
instance_info = {}

for i in tqdm(range(len(training_set))):
    item = training_set.get_txt(i)
    masks = item['label'][0].data.numpy()
    
    H,W = masks.shape
    boxes = []
    labels = []
    num_obj = 0
    
    mask_tensor = []
    areas = []
    
    for r in room_ids:
        x = copy.copy(masks)
        x[masks != r] = 0 
        x = x.astype(np.uint8)
        #plt.imshow(x)
        #plt.show()
        contours, _ = cv2.findContours(x,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        for tcnt in contours: 
            
            img = np.zeros((H,W,3), np.uint8)
            img = cv2.drawContours(img, [tcnt], -1, (255,255,255), -1)
            mask_tensor.append((img[:,:,0]/255).astype(np.int8))
            areas.append(cv2.contourArea(tcnt,False) )
            x,y,w,h = cv2.boundingRect(tcnt)
            boxes.append([x,y,x+w,y+h])
            labels.append(room_labels[room_classes[r]])
            num_obj+=1
    
    boxes = torch.FloatTensor(boxes)
    labels = torch.FloatTensor(labels).int()
    image_id = torch.ones(1, dtype=torch.uint8)*i
    areas = torch.FloatTensor(areas)
    
    try:
        mask_tensor = torch.from_numpy(np.stack(mask_tensor, 0))
    except:
        mask_tensor = torch.FloatTensor([])
    
        
    
    instance_info[item['folder']] ={'boxes': boxes, 'labels':labels, 'image_id':image_id,
                                'area':areas, 'iscrowd':torch.zeros(num_obj,dtype=torch.uint8) ,\
                                'masks': mask_tensor}
    
    
    
    
#save the file
torch.save(instance_info, 'data/cubicasa5k/instance_info_val.pt')

